### Installing packages and pulling the model
This cell runs shell commands that install the MONAI package (where all models are stored), then download the lung_nodule_ct_detection model specifically.  The model and its accompanying files will be downloaded to the /bundles/lung_nodle_ct_detection/ subdirectory.

In [ ]:
!pip install monai[fire]
!python -m monai.bundle download "lung_nodule_ct_detection" --bundle_dir "bundles/"  # model and accompanying files will be in the /bundles folder

2026-02-17 11:23:40,083 - INFO - --- input summary of monai.bundle.scripts.download ---
2026-02-17 11:23:40,083 - INFO - > name: 'lung_nodule_ct_detection'
2026-02-17 11:23:40,083 - INFO - > bundle_dir: 'bundles/'
2026-02-17 11:23:40,083 - INFO - > source: 'monaihosting'
2026-02-17 11:23:40,083 - INFO - > remove_prefix: 'monai_'
2026-02-17 11:23:40,083 - INFO - > progress: True
2026-02-17 11:23:40,083 - INFO - ---





Fetching 20 files: 100%|██████████| 20/20 [00:00<00:00, 1849.91it/s]


----------------------------------------
### Loading the model
MONAI provides a "config" file that tells you how to work with the model.  Specifies:
1. Model architecture
2. Data preprocessing specifications
3. Inference logic - the model wants a patch of the 3D image of size 512x512x192
4. Output format - how the model will output its prediction (coordinates of a bounding box surrounding the tumor)

So, what we will do is first parse the config file and get this important info.  Then, we can build the architecture specified in the config file to instantiate the model.  Finally, we load the trained weights into the architecture we instantiated.

In [1]:
import os
import torch
from monai.bundle import ConfigParser

# !!! YOU NEED TO CHANGE THIS TO YOUR ROOT PATH !!!
bundle_root = r"C:\Users\eocon\Documents\scalable project\CS6423_knowledge_distillation_project\bundles\lung_nodule_ct_detection" 

#  defines paths to the bundle and config file
config_file = os.path.join(bundle_root, "configs", "inference.json")
model_path = os.path.join(bundle_root, "models", "model.pt")

# parses the config file
parser = ConfigParser()
parser.read_config(config_file)

# defines a new element in the parsed config file - the path to our bundle folder
parser["bundle_root"] = bundle_root

#print(parser)  # see the dictionary inside if you want

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# instantiates the network - ie, creates a skeleton architecture as described in the config file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = parser.get_parsed_content("network_def").to(device)

# loads the trained weights into the skeleton architecture
network.load_state_dict(torch.load(model_path, map_location=device))
network.eval()

# load the preprocessing transforms
# this is a series of transforms that loads the image, confirms shape, normalizes, etc
preprocessing = parser.get_parsed_content("preprocessing")

print("Model and preprocessing pipeline ready.")
#print(preprocessing)

Model and preprocessing pipeline ready.


monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.


--------------------------------
### Investigating the model
We need to be aware of the actual architecture we're working with, number of parameters, etc.  These cells will output information on this.  We will first look at the named sub-networks that are within the model.  These are:
1. feature_extractor: extracts info from the scan, turning the raw input into meaningful features
2. classification_head: this will look at the extracted features and classify "is there a nodule/tumor"
3. regression_head: this will actually define a bounding box around the nodule/tumor, "where exactly is the nodule"

We'll look at the parameters and layers in each.

In [3]:
# the model has named sub-networks within - the feature_extractor, classification_head, and regression_head.  
for name, child in network.named_children():
    print(name)

total_params = sum(p.numel() for p in network.parameters())
print(f"\nTotal Parameters in the entire Lung Nodule Detector: {total_params:,}")

feature_extractor
classification_head
regression_head

Total Parameters in the entire Lung Nodule Detector: 20,902,741


In [4]:
# iterates through the sub-networks and shows how many parameters are in each
def print_detection_summary(model):
    print(f"{'Component':<25} | {'Type':<20} | {'Parameters':<15}")
    print("-" * 65)
    
    # 2. Feature Extractor (FPN)
    params = sum(p.numel() for p in model.feature_extractor.parameters())
    print(f"{'Feature Extractor':<25} | {'FPN':<20} | {params:,}")
    
    # 3. Detection Heads
    # RetinaNet has a class_head and a box_head
    params_class = sum(p.numel() for p in model.classification_head.parameters())
    params_box = sum(p.numel() for p in model.regression_head.parameters())
    print(f"{'Classification Head':<25} | {'Sub-network':<20} | {params_class:,}")
    print(f"{'Regression Head':<25} | {'Sub-network':<20} | {params_box:,}")
    
    total_params = sum(p.numel() for p in model.parameters())
    print("-" * 65)
    print(f"{'TOTAL':<25} | {'':<20} | {total_params:,}")

print_detection_summary(network)

Component                 | Type                 | Parameters     
-----------------------------------------------------------------
Feature Extractor         | FPN                  | 6,595,648
Classification Head       | Sub-network          | 7,101,699
Regression Head           | Sub-network          | 7,205,394
-----------------------------------------------------------------
TOTAL                     |                      | 20,902,741


In [5]:
# To see the types of layers in the feature extractor:
print("--- Feature Extractor ---")
print(network.feature_extractor)

--- Feature Extractor ---
BackboneWithFPN(
  (body): IntermediateLayerGetter(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [6]:
# To see the Classification Head architecture:
print("--- Classification Head ---")
print(network.classification_head)

# To see the Regression Head architecture:
print("\n--- Regression Head ---")
print(network.regression_head)

--- Classification Head ---
RetinaNetClassificationHead(
  (conv): Sequential(
    (0): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): GroupNorm(8, 256, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): GroupNorm(8, 256, eps=1e-05, affine=True)
    (5): ReLU()
    (6): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (7): GroupNorm(8, 256, eps=1e-05, affine=True)
    (8): ReLU()
    (9): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (10): GroupNorm(8, 256, eps=1e-05, affine=True)
    (11): ReLU()
  )
  (cls_logits): Conv3d(256, 3, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)

--- Regression Head ---
RetinaNetRegressionHead(
  (conv): Sequential(
    (0): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): GroupNorm(8, 256, eps=1e-05, affine=True)
 

-------------------------
### Running Inference on example data
Now we need to actually load some example data and run it through the model.  From this, we can see how our laptops handle the model, inference time, etc. to start benchmarking.

In [7]:
## NOT WORKING YET

# Replace with your actual image path
input_image = "IMG_0002.nii.gz"
parser.set("dataset#data", [{"image": input_image}])

# Initialize the inference pipeline
inferer = parser.get_parsed_content("inferer")

# Execute
with torch.no_grad():
    # Note: Lung detection often requires specific preprocessing (resampling to 0.7x0.7x1.25mm)
    # The 'preprocessing' transform in the config handles this.
    preprocessing = parser.get_parsed_content("preprocessing")
    data = preprocessing({"image": input_image})
    output = inferer(inputs=data["image"].unsqueeze(0), network=network)
    
    print("Detected Boxes:", output)

ModuleNotFoundError: Cannot locate class or function path: 'scripts.detection_inferer.RetinaNetInferer'.

### Attempts to load data:

In [1]:
from tcia_utils import nbia

# LIDC-IDRI collection name
collection = "LIDC-IDRI"

# Get a list of series for the first patient
series_data = nbia.getSeries(collection=collection, patientId="LIDC-IDRI-0001")

# Download the first CT scan series (this will download a folder of DICOMs)
# 'number=1' ensures we only get one scan for testing
nbia.downloadSeries(series_data, number=1, path="./test_data")

2026-02-17 10:50:29,870:INFO:Calling getSeries with parameters {'Collection': 'LIDC-IDRI', 'PatientID': 'LIDC-IDRI-0001'}
2026-02-17 10:50:32,972:INFO:Directory './test_data' already exists.
2026-02-17 10:50:32,973:WARNING:Series 1.3.6.1.4.1.14519.5.2.1.6279.6001.141365756818074696859567662357 already downloaded and unzipped.
2026-02-17 10:50:32,973:INFO:Found 0 previously downloaded series.
2026-02-17 10:50:32,973:INFO:Attempting to download 1 new series.
2026-02-17 10:50:32,980:INFO:Downloading... https://nbia.cancerimagingarchive.net/nbia-api/services/v4/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636438705059720603192
2026-02-17 10:51:48,961:INFO:Downloaded 1 out of 1 targeted series.
0 failed to download.
0 were previously downloaded.


In [2]:
import dicom2nifti
import os

dicom_directory = "C:/Users/eocon/Documents/scalable project/test_data/1.3.6.1.4.1.14519.5.2.1.6279.6001.141365756818074696859567662357"
output_file = "./sample_lung_ct.nii.gz"

files = os.listdir(dicom_directory)
print(f"Found {len(files)} files in directory.")
print(f"First 3 files: {files[:3]}")

'''dicom2nifti.dicom_series_to_nifti(dicom_directory, output_file)
print(f"Conversion complete: {output_file}")'''

Found 3 files in directory.
First 3 files: ['1-1.dcm', '1-2.dcm', 'LICENSE']


'dicom2nifti.dicom_series_to_nifti(dicom_directory, output_file)\nprint(f"Conversion complete: {output_file}")'